<a href="https://colab.research.google.com/github/kevinyang372/Prolog_Expert_System/blob/master/LBA_v2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
# !sudo apt install swi-prolog

In [0]:
#  pip install pyswip

In [0]:
import pandas as pd
import numpy as np

In [0]:
# Loading the data and renaming the columns 
# The names now correspond to the statement types
 
data = pd.read_csv('/content/CS152 LBA data - Sheet1.csv', skiprows=1)
col_names = ['name', 'hasCuisine', 'priceRange', 'mealTypes', 'distance',
             'isVegetarian', 'glutenFree', 'hasLanguage', 'transportation']
data.columns = col_names

In [35]:
data.head(5)

,name,hasCuisine,priceRange,mealTypes,distance,isVegetarian,glutenFree,hasLanguage,transportation
0,Miss Saigon Restaurant,Vietnamese,low,"Lunch, Dinner",17,Y,Unknown,"German, English, Vietnamese","Bus, Taxi"
1,Maroush,"Lebanese, Middle Eastern",low,"Lunch, Dinner",9,Y,Unknown,"German, English, Arabic",Walk
2,Burgermeister Kottbusser Tor,American,low,"Lunch, Dinner",12,Y,Unknown,"German, English",Walk
3,Basmah,"Sudanese, Mediterranean",low,"Lunch, Dinner",15,Y,Unknown,"German, English, Arabic","Walk, Bus"
4,Mundvoll,International,medium,"Breakfast, Brunch, Lunch, Dinner",6,Y,Y,"German, English",Walk


### Building the KB

In [0]:
# Converted Euro signs to three price levels: low, medium, high in Excel 

# all_commands = []
listCommands = []

for k in range(len(data)):
  row = data.loc[k, :]
  resName = row[0]
  # concatenate words in the name, converted to lowercase (can fix to join with lower dash later)
  resName = ''.join(e for e in resName.lower() if e.isalnum()) 
  # listCommands = [] # all commands per single restaurant name
  for i in range(1, len(row)):
    category = data.columns[i]
    items = row[i]
    if type(items) == str:
      items = row[i].split(', ')
      for item in items:
        item = ''.join(e for e in item.lower() if e.isalnum())
        command = "{0}({1}, {2})".format(category, resName, item)
        listCommands.append(command)
    else:
        command = "{0}({1}, {2})".format(category, resName, items)
        listCommands.append(command)
  # all_commands.append(listCommands) 

In [37]:
# Checking to see how Middle eastern got converted

print(listCommands[12:14])

['hasCuisine(maroush, lebanese)', 'hasCuisine(maroush, middleeastern)']


In [0]:
from pyswip import Prolog
prolog = Prolog()

# Assert KB inputs for all commands
for command in listCommands:
  prolog.assertz(command)

### Queries

In [0]:
# Includes all the variables apart from Language

prolog.assertz("query(X, Cuisine, Price, Distance, Vegetarian, Mealtype, Glutenfree, Language, Transport) :- hasCuisine(X, Cuisine), priceRange(X, Price), distance(X, Y), Y < Distance, isVegetarian(X, Vegetarian), glutenFree(X, Glutenfree), mealTypes(X, Mealtype), hasLanguage(X, Language), transportation(X, Transport)")

In [0]:
# Latest update: changed distance parameter default value to 1000 from -1

def search_new(cuisine = '_', price = '_', distance = '1000', isvegetarian = '_', mealtype = '_', isglutenfree = '_', language = '_' , transport = '_'):
  to_query = "query(X, %s)" % (",".join([cuisine, price, distance, isvegetarian, mealtype, isglutenfree, language, transport]))
  return set([soln["X"] for soln in prolog.query(to_query)])

### Test cases

In [50]:
# Search case 1: All restaurants within a 15 min walking distance

search_new(distance = "15")

{'burgermeisterkottbussertor',
 'hasirburger',
 'lapiadina',
 'longmarchcanteen',
 'maroush',
 'milchzucker',
 'mundvoll',
 'paglia',
 'quepasamexicana',
 'santamaria',
 'sushiforyou'}

In [48]:
# Search case 2: A restaurant within a 10 min walking distance in a low price range
# that has English-speaking staff, vegetarian and dinner options

search_new(distance = "10", price="low", isvegetarian="y", mealtype="dinner", language = 'english', transport="walk")

{'lapiadina', 'maroush'}

In [51]:
# Search case 3: Filter by cuisine (italian) and price (medium)

search_new(cuisine = 'italian', price="medium")

{'paglia'}

In [52]:
# Search case 4: Group dinner search; prefer American cuisine in a low price range and near the residence

search_new(cuisine = 'american', distance = "15", price="low")

{'burgermeisterkottbussertor'}

In [24]:
from ipywidgets import interact
interact(search_new, cuisine = "_", price = "_", distance = "1000", isvegetarian = "_", mealtype = '_', isglutenfree = '_', transport = '_')
# interact(search, cuisine = "_", price = "_", distance = "-1")

interactive(children=(Text(value='_', description='cuisine'), Text(value='_', description='price'), Text(value…

<function __main__.search_new>